# Imports

In [1]:
import os
import json
import yaml
import numpy as np
import pandas as pd

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

# Variables

In [2]:
path_pdf = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\cencora_earning_reports\AmerisourceBergen-Reports-Fiscal-2023-Second-Quarter-Results-2023.pdf"

embed_model= "text-embedding-ada-002"
vectore_store_persist_directory = 'vector_store'

# Config

In [3]:
import pandas as pd

import sys
import os
sys.path.append(os.path.expanduser("~/.utils"))

from util_openai import get_config_dict

config = get_config_dict(env=None)
config['openai_api_key'][:4]

'f400'

# Load pdf

In [4]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(path_pdf)
documents = loader.load()

# Split Documents Into Chunks

In [5]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

# Create Embeddings

In [6]:
from langchain.embeddings import AzureOpenAIEmbeddings

embed_model= "text-embedding-ada-002"
embeddings = AzureOpenAIEmbeddings(
    model=embed_model,
    chunk_size=1,
    openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86",
    azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev"
    )
embeddings

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000027814182A90>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x000002781415B8D0>, model='text-embedding-ada-002', deployment=None, openai_api_version='2023-05-15', openai_api_base=None, openai_api_type='azure', openai_proxy='', embedding_ctx_length=8191, openai_api_key='242ad9b0c6d84b5fb4307061d5f05a86', openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, azure_endpoint='https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev', azure_ad_token=None, azure_ad_token_provider=None, validate_base_url=True)

# Create Vector Store and Retriever

In [7]:
# openai
env = 'openai'

if env == 'openai':
    import openai

    print(f'Setting environment for: {env}\n\n')
    openai.api_key = config['openai_api_key']
    openai.api_base = config['openai_api_base']
    openai.api_type = 'azure'
    openai.api_version = config['openai_api_version']

    # set env keys
    keys = ["OPENAI_API_KEY","OPENAI_API_TYPE","OPENAI_API_BASE","OPENAI_API_VERSION"]
    for key in keys:
        os.environ[key] = config.get(key)

Setting environment for: openai




In [8]:
if 'OPENAI_API_BASE' in os.environ:
    del os.environ['OPENAI_API_BASE']

llm = AzureChatOpenAI(openai_api_type="azure",
                openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86",
                model_name="text-davinci-002",
                azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev",
                openai_api_version="2023-03-15-preview"
                  )

C:\Users\a126291\venv\py311_camelot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.azure_openai.AzureChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(


In [9]:
from langchain.vectorstores import FAISS

pages = loader.load_and_split()
vector_store = FAISS.from_documents(documents=pages, embedding=embeddings)
vector_store.save_local(vectore_store_persist_directory)

# LLM

In [10]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(openai_api_type="azure",
                openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86",
                model_name="text-davinci-002",
                azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev",
                openai_api_version="2023-03-15-preview"
                  )

# Get vector store from local file

In [11]:
from langchain.vectorstores import FAISS

vector_store = FAISS.load_local(vectore_store_persist_directory, embeddings, allow_dangerous_deserialization=True,)
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2},
    return_source_documents=True,
)

# Create RetrivalQA Chain

In [12]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = retriever
)

# Create Query And Get Response

In [13]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever = retriever
)

query = 'What are the year to year gain in revenue?'
qa.invoke({'query': query, 'chat_history': []})

NotFoundError: Error code: 404 - {'error': {'code': 'DeploymentNotFound', 'message': 'The API deployment for this resource does not exist. If you created the deployment within the last 5 minutes, please wait a moment and try again.'}}

In [ ]:
result = qa.invoke({'query': query, 'chat_history': []})
print(result['result'])

## Using: ConversationalRetrievalChain

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
qa = ConversationalRetrievalChain.from_llm(llm,retriever)

chat_history = []
query = "Is there gain or loss of revenue in this quarter?"

result = qa.invoke({'question': query, 'chat_history': chat_history})
print(result['answer'])

In [ ]:
chat_history = [(query,result['answer'])]
query = "Is there gain in revenue for U.S. Healthcare Solutions this quarter?"

result = qa.invoke({'question': query, 'chat_history': chat_history})
print(result['answer'])

# Script

In [ ]:
%%writefile a.py
# imports
import os
import json
import yaml
import numpy as np
import pandas as pd

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain

# config
def get_config_dict():
    import os
    import yaml

    with open(os.path.expanduser('~/.config/config.yaml')) as fh:
        config = yaml.safe_load(fh)

    # openai
    keys = ["OPENAI_API_KEY","OPENAI_API_TYPE","OPENAI_API_BASE","OPENAI_API_VERSION"]
    for key in keys:
        os.environ[key] = config.get(key)

    return config

config = get_config_dict()

# variables
path_pdf = r"C:\Users\a126291\OneDrive - AmerisourceBergen(ABC)\data\pdf_files\cencora_earning_reports\AmerisourceBergen-Reports-Fiscal-2023-Second-Quarter-Results-2023.pdf"

embed_model= "text-embedding-ada-002"
vectore_store_persist_directory = 'vector_store'

# load documents
loader = PyPDFLoader(path_pdf)
documents = loader.load()

# create embeddings
embed_model= "text-embedding-ada-002"
embeddings = AzureOpenAIEmbeddings(model=embed_model, chunk_size=1)

# create vector store
pages = loader.load_and_split()
vector_store = FAISS.from_documents(documents=pages, embedding=embeddings)
vector_store.save_local(vectore_store_persist_directory)

# llm
if 'OPENAI_API_BASE' in os.environ:
    del os.environ['OPENAI_API_BASE']

llm = AzureChatOpenAI(openai_api_type="azure",
                openai_api_key="242ad9b0c6d84b5fb4307061d5f05a86",
                model_name="text-davinci-002",
                azure_endpoint="https://cencoragaidev-bhakadc4hreqbthb.z01.azurefd.net/gpsdatascience_dev",
                openai_api_version="2023-03-15-preview"
                  )

# load vector store from local file
vector_store = FAISS.load_local(vectore_store_persist_directory, embeddings allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2},
    return_source_documents=True
)

# question answer
qa = ConversationalRetrievalChain.from_llm(llm, retriever)
chat_history = []
query = "Is there gain or loss of revenue in this quarter?"
result = qa({'question': query, 'chat_history': chat_history})
print(result['answer'])

# another chat
print()
chat_history = [(query,result['answer'])]
query = "Is there gain in revenue for U.S. Healthcare Solutions this quarter?"

result = qa({'question': query, 'chat_history': chat_history})
print(result['answer'])

In [ ]:
import pyperclip

pyperclip.copy(os.getcwd())

In [ ]:
# !python a.py